使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第七章第一節 給湯設備）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import numpy as np

## 付録D　石油給湯機及び石油給湯温水暖房機の給湯部  / Appendix D Water heater of oil water heater and oil water heater hot water heater

### D.1 記号及び単位 / Symbol and unit

省略

Omission

### D.2 消費電力量 / Power consumption

$$\large
E_{E,hs,d,t} = E_{E,hs,aux1,d,t} + E_{E,hs,aux2,d,t} + E_{E,hs,aux3,d,t}
$$

$E_{E,hs,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の消費電力量(kWh/h) / Water heater power consumption per hour at time $ t $ on date $ d $  
$E_{E,hs,aux1,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の待機時及び水栓給湯時の補機による消費電力量(kWh/h) / Power consumption by the auxiliary unit at the time of standby of the water heater and hourly water supply at the time $ t $ of date $ d $  
$E_{E,hs,aux2,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の湯はり時の補機による消費電力量(kWh/h) / Power consumption by auxiliary equipment at the time of hot water heating of water heater per hour at time $ t $ of date $ d $  
$E_{E,hs,aux3,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の保温時の補機による消費電力量(kWh/h) / Power consumption by auxiliary equipment at the time of hot water heater warm-up per hour at time $ t $ of date $ d $  

In [ ]:
# 1時間当たりの給湯機の消費電力量 (1)
def get_E_E_hs_d_t(W_dash_k_d_t, W_dash_s_d_t, W_dash_w_d_t, W_dash_b1_d_t, W_dash_ba1_d_t, W_dash_b2_d_t,
                   Theta_ex_Ave_d, L_dashdash_ba2_d_t):
    """
    :param W_dash_k_d_t: 1日当たりの台所水栓における節湯補正給湯量 (L/h)
    :param W_dash_s_d_t: 1日当たりの浴室シャワー水栓における節湯補正給湯量 (L/h)
    :param W_dash_w_d_t: 1日当たりの洗面水栓における節湯補正給湯量 (L/h)
    :param W_dash_b1_d_t: 1日当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/h)
    :param W_dash_ba1_d_t: 1日当たりの浴槽水栓さし湯時における節湯補正給湯量 (L/h)
    :param W_dash_b2_d_t: 1日当たりの浴槽追焚時における節湯補正給湯量 (L/h)
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_ba2_d_t: 1日当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/h)
    :return: 1時間当たりの給湯機の消費電力量 (kWh/h)
    """
    # 待機時及び水栓給湯時の補機による消費電力量 (2)
    E_E_hs_aux1_d_t = get_E_E_hs_aux1_d_t(W_dash_k_d_t, W_dash_s_d_t, W_dash_w_d_t, W_dash_b1_d_t, W_dash_ba1_d_t,
                                          Theta_ex_Ave_d)

    # 湯はり時の補機による消費電力量 (3)
    E_E_hs_aux2_d_t = get_E_E_hs_aux2_d_t(W_dash_b2_d_t)

    # 保温時の補機による消費電力量 (4)
    E_E_hs_aux3_d_t = get_E_E_hs_aux3_d_t(L_dashdash_ba2_d_t)

    # 1日当たりの給湯機の消費電力量 (1)
    E_E_hs_d_t = E_E_hs_aux1_d_t + E_E_hs_aux2_d_t + E_E_hs_aux3_d_t

    return E_E_hs_d_t

$$\large
E_{E,hs,aux1,d,t} = \big\{ \big( -0.00235 \times \theta_{ex,Ave,d} + 0.3388 \big) \div 24\\
\large + 0.000780 \times \big(W'_{k,d,t}+W'_{s,d,t}+W'_{w,d,t}+W'_{b1,d,t}+W'_{ba1,d,t}\big) \big\} \times 10^{3} \div 3600
$$

$E_{E,hs,aux1,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の待機時及び水栓給湯時の補機による消費電力量(kWh/h) / Power consumption by the auxiliary unit at the time of standby of the water heater and hourly water supply at the time $ t $ of date $ d $  
$ W'_{k,d,t} $：日付$d$の時刻$t$における 1時間当たりの台所水栓における節湯補正給湯量(L/h) / Hot water correction amount of hot water at kitchen faucet per hour at time $ t $ on date $ d $  
$ W'_{s,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴室シャワー水栓における節湯補正給湯量(L/h) / Hot water correction amount of hot water in bathroom shower faucet per hour at time $ t $ of date $ d $  
$ W'_{w,d,t} $：日付$d$の時刻$t$における 1時間当たりの洗面水栓における節湯補正給湯量(L/h) / Hot water correction amount of hot water in the faucet per hour at time $ t $ of date $ d $  
$ W'_{b1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓湯はり時における節湯補正給湯量(L/h) / Hot water correction hot-water supply amount at the time of bathtub $ 1 $ per hour at date $ t $ on date $ d $  
$ W'_{ba1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓さし湯時における節湯補正給湯量(L/h) / Hot water correction hot-water supply amount at the time of $ t $ for the time $ t $ on the date $ t $  
$\theta_{ex,Ave,d}$：日付$d$における日平均外気温度(℃) / Daily average outside temperature on date $ d $  

In [ ]:
# 1時間当たりの給湯機の待機時及び水栓給湯時の補機による消費電力量 (2)
def get_E_E_hs_aux1_d_t(W_dash_k_d_t, W_dash_s_d_t, W_dash_w_d_t, W_dash_b1_d_t, W_dash_ba1_d_t, Theta_ex_Ave_d):
    """
    :param W_dash_k_d_t: 1日当たりの台所水栓における節湯補正給湯量 (L/h)
    :param W_dash_s_d_t: 1日当たりの浴室シャワー水栓における節湯補正給湯量 (L/h)
    :param W_dash_w_d_t: 1日当たりの洗面水栓における節湯補正給湯量 (L/h)
    :param W_dash_b1_d_t: 1日当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/h)
    :param W_dash_ba1_d_t: 1日当たりの浴槽水栓さし湯時における節湯補正給湯量 (L/h)
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :return: 1時間当たりの給湯機の待機時及び水栓給湯時の補機による消費電力量 (kWh/h)
    """
    # 1時間当たりの給湯機の待機時及び水栓給湯時の補機による消費電力量 (2)
    E_E_hs_aux1_d_t = ((-0.00235 * np.repeat(Theta_ex_Ave_d, 24) + 0.3388) / 24
                       + 0.000780 * (
                               W_dash_k_d_t + W_dash_s_d_t + W_dash_w_d_t + W_dash_b1_d_t + W_dash_ba1_d_t)) * 10 ** 3 / 3600

    return E_E_hs_aux1_d_t

$$\large
E_{E,hs,aux2,d,t} = \left\{ \begin{array}{ll}
    0.07 \times 10^{3} \div 3600 \times \frac{W'_{b2,d,t}}{\sum_{t=0}^{23} W'_{b2,d,t}} & \big(\sum_{t=0}^{23} W'_{b2,d,t} > 0 \big)\\ 
    0 & \big(\sum_{t=0}^{23} W'_{b2,d,t} = 0 \big)\\ 
    \end{array}
\right.
$$

$E_{E,hs,aux2,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の湯はり時の補機による消費電力量(kWh/h) / Power consumption by auxiliary equipment at the time of hot water heating of water heater per hour at time $ t $ of date $ d $  
$W'_{b2,d,t}$：日付$d$の時刻$t$における 1時間当たりの浴槽自動湯はり時における節湯補正給湯量(L/h) / Hot water correction hot water supply amount at the time of bathtub automatic water heating per hour at time $ t $ of date $ d $  

In [ ]:
# 1時間当たりの給湯機の湯はり時の補機による消費電力量 (3)
def get_E_E_hs_aux2_d_t(W_dash_b2_d_t):
    """
    :param W_dash_b2_d_t: 1日当たりの浴槽追焚時における節湯補正給湯量 (L/h)
    :return: 1時間当たりの給湯機の湯はり時の補機による消費電力量 (kWh/h)
    """
    E_E_hs_aux2_d_t = np.zeros(24 * 365)

    # 1日ごとにまとめる
    W_dash_b2_d = np.repeat(np.sum(W_dash_b2_d_t.reshape(365, 24), axis=1), 24)

    # W_dash_b2_d > 0 の場合
    f = W_dash_b2_d > 0
    E_E_hs_aux2_d_t[f] = (0.07 * 10 ** 3 / 3600) * W_dash_b2_d_t[f] / W_dash_b2_d[f]

    return E_E_hs_aux2_d_t

$$\large
E_{E,hs,aux3,d,t} = \left\{ \begin{array}{ll}
    (0.02102 \times L''_{ba2,d} + 0.12852) \times 10^{3} \div 3600 \times \frac{L''_{ba2,d,t}}{L''_{ba2,d}} & \big(L''_{ba2,d} > 0 \big)\\ 
    0 & \big(L''_{ba2,d} = 0 \big)\\ 
    \end{array}
\right.
$$

$E_{E,hs,aux3,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の保温時の補機による消費電力量(kWh/h) / Power consumption by auxiliary equipment at the time of hot water heater warm-up per hour at time $ t $ of date $ d $  
$ L''_{ba2,d} $：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub remembrance at date $ d $  
$ L''_{ba2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of tub recollection per hour at time $ t $ of date $ d $  

In [ ]:
# 1時間当たりの給湯機の保温時の補機による消費電力量 (4)
def get_E_E_hs_aux3_d_t(L_dashdash_ba2_d_t):
    """
    :param L_dashdash_ba2_d_t: 1日当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 1日当たりの給湯機の保温時の補機による消費電力量 (kWh/d)
    """
    E_E_hs_aux3 = np.zeros(24 * 365)

    L_dashdash_ba2_d = get_L_dashdash_ba2_d(L_dashdash_ba2_d_t)
    L_dashdash_ba2_d = np.repeat(L_dashdash_ba2_d, 24)

    f = (L_dashdash_ba2_d > 0)
    E_E_hs_aux3[f] = (0.02102 * L_dashdash_ba2_d[f] + 0.12852) * 10 ** 3 / 3600 \
                     * L_dashdash_ba2_d_t[f] / L_dashdash_ba2_d[f]

    return E_E_hs_aux3


### D.3 ガス消費量 / Gas consumption

日付$d$の時刻$t$における 1時間当たりの給湯機のガス消費量$E_{G,hs,d,t}$は0とする。

The gas consumption $ E_ {G, hs, d, t} $ of the water heater per hour at the time $ t $ of the date $ d $ is 0.

In [ ]:
# 1日当たりの給湯機のガス消費量
def get_E_G_hs_d_t():
    # 1日当たりの給湯機のガス消費量は0とする
    return np.zeros(24 * 365)

### D.4 灯油消費量 / Kerosene consumption

給湯単機能の場合

In the case of the hot water supply single function

$$\large
E_{K,hs,d,t} = \frac{L''_{k,d,t}}{e_{k,d}} + \frac{L''_{s,d,t}}{e_{s,d}} + \frac{L''_{w,d,t}}{e_{w,d}} + \frac{L''_{b1,d,t}}{e_{b1,d}} + \frac{L''_{ba1,d,t}}{e_{ba1,d}}
$$

ふろ給湯機（追焚なし）の場合

In the case of bath heater (without follow-up)

$$\large
E_{K,hs,d,t} = \frac{L''_{k,d,t}}{e_{k,d}} + \frac{L''_{s,d,t}}{e_{s,d}} + \frac{L''_{w,d,t}}{e_{w,d}} + \frac{L''_{b2,d,t}}{e_{b2,d}} + \frac{L''_{ba1,d,t}}{e_{ba1,d}}
$$

ふろ給湯機（追焚あり）の場合

In the case of bath heater (with a memorial service)

$$\large
E_{K,hs,d,t} = \frac{L''_{k,d,t}}{e_{k,d}} + \frac{L''_{s,d,t}}{e_{s,d}} + \frac{L''_{w,d,t}}{e_{w,d}} + \frac{L''_{b2,d,t}}{e_{b2,d}} + \frac{L''_{ba2,d,t}}{e_{ba2,d}}
$$

$E_{K,hs,d,t}$：日付$d$の時刻$t$における 1時間当たりの給湯機の灯油消費量(MJ/h) / Water heater kerosene consumption per hour at time $ t $ on date $ d $  
$ L''_{k,d,t}$：日付$d$の時刻$t$における1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load on kitchen faucet per hour at time $ t $ of date $ d $  
$ L''_{s,d,t}$：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load in bathroom shower faucet per hour at time $ t $ of date $ d $  
$ L''_{w,d,t}$：日付$d$の時刻$t$における1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load in the faucet per hour at time $ t $ of date $ d $  
$ L''_{b1,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub tap water heating per hour at time $ t $ of date $ d $  
$ L''_{b2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub automatic water heating per hour at time $ t $ of date $ d $  
$ L''_{ba1,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub faucet breaking water per hour at time $ t $ of date $ d $   
$ L''_{ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of tub recollection per hour at time $ t $ of date $ d $  
$ e_{k,d} $：日付$d$における台所水栓の給湯使用時における日平均給湯機効率(-) / Daily average water heater efficiency when using hot water of kitchen faucet on date $ d $  
$ e_{s,d} $：日付$d$における浴室シャワー水栓の給湯使用時における日平均給湯機効率(-) / Daily average water heater efficiency when using hot water in bathroom shower faucet with date $ d $  
$ e_{w,d} $：日付$d$における洗面水栓の給湯使用時における日平均給湯機効率(-) / Daily average water heater efficiency when using hot water supply of wash water faucet on date $ d $  
$ e_{b1,d} $：日付$d$における浴槽水栓湯はり時における日平均給湯機効率(-) / Daily average water heater efficiency at bathtub faucet hot water at date $ d $  
$ e_{b2,d} $：日付$d$における浴槽自動湯はり時における日平均給湯機効率(-) / Daily average water heater efficiency at bathtub automatic water bathing on date $ d $  
$ e_{ba1,d} $：日付$d$における浴槽水栓さし湯時における日平均給湯機効率(-) / Daily average hot water supply efficiency at the time of bathtub faucet opening at date $ d $  
$ e_{ba2,d} $：日付$d$における浴槽追焚時における日平均給湯機効率(-) / Daily average water heater efficiency at the time of bathtub remembrance on date $ d $  

In [ ]:
# 灯油消費量 (5)
def get_E_K_hs_d_t(hw_type, e_rtd, e_dash_rtd, bath_function, Theta_ex_Ave_d, L_dashdash_k_d_t, L_dashdash_s_d_t, L_dashdash_w_d_t,
                   L_dashdash_b1_d_t, L_dashdash_b2_d_t, L_dashdash_ba1_d_t, L_dashdash_ba2_d_t):
    """
    :param hw_type: 給湯機／給湯温水暖房機の種類
    :param e_rtd: 当該給湯機の効率
    :param e_dash_rtd: ：「エネルギーの使用の合理化に関する法律」に基づく「特定機器の性能の向上に関する製造事業者等の 判断の基準等」（ガス温水機器）に定義される「エネルギー消費効率」
    :param bath_function: ふろ機能の種類
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :return: 1時間当たりの給湯機の灯油消費量 (MJ/h)
    """
    # 効率の決定
    if e_rtd == None:
        if e_dash_rtd == None:
            e_rtd = get_e_rtd_default(hw_type)
        else:
            e_rtd = get_e_rtd(e_dash_rtd)

    # 1日当たりの太陽熱補正給湯熱負荷
    L_dashdash_k_d = get_L_dashdash_k_d(L_dashdash_k_d_t)
    L_dashdash_s_d = get_L_dashdash_s_d(L_dashdash_s_d_t)
    L_dashdash_w_d = get_L_dashdash_w_d(L_dashdash_w_d_t)
    L_dashdash_b1_d = get_L_dashdash_b1_d(L_dashdash_b1_d_t)
    L_dashdash_b2_d = get_L_dashdash_b2_d(L_dashdash_b2_d_t)
    L_dashdash_ba1_d = get_L_dashdash_ba1_d(L_dashdash_ba1_d_t)
    L_dashdash_ba2_d = get_L_dashdash_ba2_d(L_dashdash_ba2_d_t)

    # 日平均給湯機効率
    e_k_d = get_e_k_d(e_rtd, Theta_ex_Ave_d, L_dashdash_k_d, L_dashdash_w_d)
    e_s_d = get_e_s_d(e_rtd, Theta_ex_Ave_d, L_dashdash_s_d)
    e_w_d = get_e_w_d(e_rtd, Theta_ex_Ave_d, L_dashdash_k_d, L_dashdash_w_d)
    e_b1_d = get_e_b1_d(e_rtd, Theta_ex_Ave_d, L_dashdash_b1_d)
    e_b2_d = get_e_b2_d(e_rtd, Theta_ex_Ave_d, L_dashdash_b2_d)
    e_ba1_d = get_e_ba1_d(e_rtd, Theta_ex_Ave_d, L_dashdash_ba1_d)
    e_ba2_d = get_e_ba2_d(e_rtd, Theta_ex_Ave_d, L_dashdash_ba2_d)

    e_k_d = np.repeat(e_k_d, 24)
    e_s_d = np.repeat(e_s_d, 24)
    e_w_d = np.repeat(e_w_d, 24)
    e_b1_d = np.repeat(e_b1_d, 24)
    e_b2_d = np.repeat(e_b2_d, 24)
    e_ba1_d = np.repeat(e_ba1_d, 24)
    e_ba2_d = np.repeat(e_ba2_d, 24)

    if bath_function == '給湯単機能':
        # (5a)
        return L_dashdash_k_d_t / e_k_d \
               + L_dashdash_s_d_t / e_s_d \
               + L_dashdash_w_d_t / e_w_d \
               + L_dashdash_b1_d_t / e_b1_d \
               + L_dashdash_ba1_d_t / e_ba1_d
    elif bath_function == 'ふろ給湯機(追焚なし)':
        # (5b)
        return L_dashdash_k_d_t / e_k_d \
               + L_dashdash_s_d_t / e_s_d \
               + L_dashdash_w_d_t / e_w_d \
               + L_dashdash_b2_d_t / e_b2_d \
               + L_dashdash_ba1_d_t / e_ba1_d
    elif bath_function == 'ふろ給湯機(追焚あり)':
        # (5c)
        return L_dashdash_k_d_t / e_k_d \
               + L_dashdash_s_d_t / e_s_d \
               + L_dashdash_w_d_t / e_w_d \
               + L_dashdash_b2_d_t / e_b2_d \
               + L_dashdash_ba2_d_t / e_ba2_d
    else:
        raise ValueError(bath_function)


各用途の日平均給湯機効率$e_{u,d}$は1.0を超えない。

The daily average water heater efficiency $ e_ {u, d} $ for each application does not exceed 1.0.

$$\large
e_{k,d} = a_{k} \times \theta_{ex,Ave,d} + b_{k} \times (L''_{k,d} + L''_{w,d}) + c_{k}
$$

$$\large
e_{s,d} = a_{s} \times \theta_{ex,Ave,d} + b_{s} \times L''_{s,d} + c_{s}
$$

$$\large
e_{w,d} = a_{w} \times \theta_{ex,Ave,d} + b_{w} \times (L''_{k,d} + L''_{w,d}) + c_{w}
$$

$$\large
e_{b1,d} = a_{b1} \times \theta_{ex,Ave,d} + b_{b1} \times L''_{b1,d} + c_{b1}
$$

$$\large
e_{b2,d} = a_{b2} \times \theta_{ex,Ave,d} + b_{b2} \times L''_{b2,d} + c_{b2}
$$

$$\large
e_{ba1,d} = a_{ba1} \times \theta_{ex,Ave,d} + b_{ba1} \times L''_{ba1,d} + c_{ba1}
$$

$$\large
e_{ba2,d} = a_{ba2} \times \theta_{ex,Ave,d} + b_{ba2} \times L''_{ba2,d} + c_{ba2}
$$

$ e_{k,d} $：日付$d$における台所水栓の給湯使用時における日平均給湯機効率(-) / Daily average water heater efficiency when using hot water of kitchen faucet on date $ d $  
$ e_{s,d} $：日付$d$における浴室シャワー水栓の給湯使用時における日平均給湯機効率(-) / Daily average water heater efficiency when using hot water in bathroom shower faucet with date $ d $  
$ e_{w,d} $：日付$d$における洗面水栓の給湯使用時における日平均給湯機効率(-) / Daily average water heater efficiency at the time of using hot water supply of wash water faucet on date $ d $  
$ e_{b1,d} $：日付$d$における浴槽水栓湯はり時における日平均給湯機効率(-) / Daily average water heater efficiency at bathtub faucet hot water at date $ d $  
$ e_{b2,d} $：日付$d$における浴槽自動湯はり時における日平均給湯機効率(-) / Daily average water heater efficiency at bathtub automatic bathing at date $ d $  
$ e_{ba1,d} $：日付$d$における浴槽水栓さし湯時における日平均給湯機効率(-) / Daily average water heater efficiency at the time of bathtub faucet breaking water at date $ d $  
$ e_{ba2,d} $：日付$d$における浴槽追焚時における日平均給湯機効率(-) / Daily average water heater efficiency at bathtub remembrance on date $ d $  
$L''_{k,d}$：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load in a kitchen faucet per day at date $ d $  
$L''_{s,d}$：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar thermal correction hot water supply heat load at one day bathroom shower faucet at date $ d $  
$L''_{w,d}$：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load in a single day water faucet at date $ d $  
$L''_{b1,d}$：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at one day bathtub bathtub faucet at date $ d $  
$L''_{b2,d}$：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar thermal correction hot water supply heat load at one day bathtub automatic bath heating on date $ d $  
$L''_{ba1,d}$：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub bathtub faucet at date $ d $  
$L''_{ba2,d}$：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub remembrance at date $ d $  
$\theta_{ex,Ave,d}$：日付$d$における日平均外気温度(℃) / Daily average outside temperature on date $ d $  
$a_{u}, b_{u}, c_{u}$：回帰係数(-) / Regression coefficient  
$u$：用途を表す添え字($k,s,w,b1,b2,ba1,ba2$) / Index indicating use

$$\large
a_{u} = a_{std,u} \times f_{hs}
$$

$$\large
b_{u} = b_{std,u} \times f_{hs}
$$

$$\large
c_{u} = c_{std,u} \times f_{hs}
$$

$a_{u}, b_{u}, c_{u}$：回帰係数(-) / Regression coefficient  
$a_{std,u}, b_{std,u}, c_{std,u}$：表により求まる係数(-) / Coefficient determined by table  
$f_{hs}$：当該給湯機に対する効率の補正係数(-) / Correction factor of efficiency for the relevant water heater    
$u$：用途を表す添え字（$k, s, w, b1, b2, ba1, ba2$) / Index indicating use   

In [ ]:
# 台所水栓の給湯使用時における日平均給湯機効率 (6a)
def get_e_k_d(e_rtd, Theta_ex_Ave_d, L_dashdash_k_d, L_dashdash_w_d):
    """
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_k_d: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_w_d: 1日当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 台所水栓の給湯使用時における日平均給湯機効率
    """
    # 当該給湯機に対する効率の補正係数
    f_hs = get_f_hs(e_rtd)

    # 石油給湯機効率の回帰係数
    a_std_k = table_d_3()[0][0]
    b_std_k = table_d_3()[1][0]
    c_std_k = table_d_3()[2][0]
    a_k = a_std_k * f_hs  # (7a)
    b_k = b_std_k * f_hs  # (7b)
    c_k = c_std_k * f_hs  # (7c)

    e_k = a_k * Theta_ex_Ave_d + b_k * (L_dashdash_k_d + L_dashdash_w_d) + c_k

    return np.clip(e_k, 0.0, 1.0)


# 浴室シャワー水栓の給湯使用時における日平均給湯機効率 (6b)
def get_e_s_d(e_rtd, Theta_ex_Ave_d, L_dashdash_s_d):
    """
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_s_d: 1日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 浴室シャワー水栓における日平均給湯機効率
    """
    # 当該給湯機に対する効率の補正係数
    f_hs = get_f_hs(e_rtd)

    # 石油給湯機効率の回帰係数
    a_std_s = table_d_3()[0][1]
    b_std_s = table_d_3()[1][1]
    c_std_s = table_d_3()[2][1]
    a_s = a_std_s * f_hs  # (7a)
    b_s = b_std_s * f_hs  # (7b)
    c_s = c_std_s * f_hs  # (7c)

    e_s = a_s * Theta_ex_Ave_d + b_s * L_dashdash_s_d + c_s

    return np.clip(e_s, 0.0, 1.0)


# 洗面水栓の給湯使用時における日平均給湯機効率 (6c)
def get_e_w_d(e_rtd, Theta_ex_Ave_d, L_dashdash_k_d, L_dashdash_w_d):
    """
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_k_d: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_w_d: 1日当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 洗面水栓の給湯使用時における日平均給湯機効率
    """
    # 当該給湯機に対する効率の補正係数
    f_hs = get_f_hs(e_rtd)

    # 石油給湯機効率の回帰係数
    a_std_w = table_d_3()[0][2]
    b_std_w = table_d_3()[1][2]
    c_std_w = table_d_3()[2][2]
    a_w = a_std_w * f_hs  # (7a)
    b_w = b_std_w * f_hs  # (7b)
    c_w = c_std_w * f_hs  # (7c)

    e_w = a_w * Theta_ex_Ave_d + b_w * (L_dashdash_k_d + L_dashdash_w_d) + c_w

    return np.clip(e_w, 0.0, 1.0)


# 浴槽水栓湯はり時における日平均給湯機効率 (6d)
def get_e_b1_d(e_rtd, Theta_ex_Ave_d, L_dashdash_b1_d):
    """
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_b1_d: 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 浴槽水栓湯はり時における日平均給湯機効率
    """
    # 当該給湯機に対する効率の補正係数
    f_hs = get_f_hs(e_rtd)

    # 石油給湯機効率の回帰係数
    a_std_b1 = table_d_3()[0][3]
    b_std_b1 = table_d_3()[1][3]
    c_std_b1 = table_d_3()[2][3]
    a_b1 = a_std_b1 * f_hs  # (7a)
    b_b1 = b_std_b1 * f_hs  # (7b)
    c_b1 = c_std_b1 * f_hs  # (7c)

    e_b1 = a_b1 * Theta_ex_Ave_d + b_b1 * L_dashdash_b1_d + c_b1

    return np.clip(e_b1, 0.0, 1.0)


# 浴槽自動湯はり時における日平均給湯機効率 (6e)
def get_e_b2_d(e_rtd, Theta_ex_Ave_d, L_dashdash_b2_d):
    """
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_b2_d: 1日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 浴槽自動湯はり時における日平均給湯機効率
    """
    # 当該給湯機に対する効率の補正係数
    f_hs = get_f_hs(e_rtd)

    # 石油給湯機効率の回帰係数
    a_std_b2 = table_d_3()[0][4]
    b_std_b2 = table_d_3()[1][4]
    c_std_b2 = table_d_3()[2][4]
    a_b2 = a_std_b2 * f_hs  # (7a)
    b_b2 = b_std_b2 * f_hs  # (7b)
    c_b2 = c_std_b2 * f_hs  # (7c)

    e_b2 = a_b2 * Theta_ex_Ave_d + b_b2 * L_dashdash_b2_d + c_b2

    return np.clip(e_b2, 0.0, 1.0)


# 浴槽水さし時における日平均給湯機効率 (6f)
def get_e_ba1_d(e_rtd, Theta_ex_Ave_d, L_dashdash_ba1_d):
    """
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_ba1_d: 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 浴槽水さし時における日平均給湯機効率
    """
    # 当該給湯機に対する効率の補正係数
    f_hs = get_f_hs(e_rtd)

    # 石油給湯機効率の回帰係数
    a_std_ba1 = table_d_3()[0][5]
    b_std_ba1 = table_d_3()[1][5]
    c_std_ba1 = table_d_3()[2][5]
    a_ba1 = a_std_ba1 * f_hs  # (7a)
    b_ba1 = b_std_ba1 * f_hs  # (7b)
    c_ba1 = c_std_ba1 * f_hs  # (7c)

    e_ba1 = a_ba1 * Theta_ex_Ave_d + b_ba1 * L_dashdash_ba1_d + c_ba1

    return np.clip(e_ba1, 0.0, 1.0)


# 浴槽追焚時における日平均給湯機効率 (6g)
def get_e_ba2_d(e_rtd, Theta_ex_Ave_d, L_dashdash_ba2_d):
    """
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_ba2_d: 1日当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/d)
    :return: 浴槽追焚時における日平均給湯機効率
    """
    # 当該給湯機に対する効率の補正係数
    f_hs = get_f_hs(e_rtd)

    # 石油給湯機効率の回帰係数
    a_std_ba2 = table_d_3()[0][6]
    b_std_ba2 = table_d_3()[1][6]
    c_std_ba2 = table_d_3()[2][6]
    a_ba2 = a_std_ba2 * f_hs  # (7a)
    b_ba2 = b_std_ba2 * f_hs  # (7b)
    c_ba2 = c_std_ba2 * f_hs  # (7c)

    e_ba2 = a_ba2 * Theta_ex_Ave_d + b_ba2 * L_dashdash_ba2_d + c_ba2

    return np.clip(e_ba2, 0.0, 1.0)

<div style="text-align: center;font-weight: bold;">表　石油給湯機効率の回帰係数$a_{std,u}$、$b_{std,u}$及び$c_{std,u}$<br>Table: Regression coefficients $ a_ {std, u} $, $ b_ {std, u} $ and $ c_ {std, u} $ of oil water heater efficiency</div>

<table>
<thead>
<tr>
<th rowspan="2">回帰係数<br>Regression coefficient</th>
<th colspan="7" style="text-align: center;">添え字 $u$ （用途）<br>Subscript $ u $ (use)</th>
</tr>
<tr>
<th>台所水栓<br> $k$<br>Kitchen faucet</th>
<th>浴室<br>シャワー水栓<br> $s$<br>Bathroom shower faucet</th>
<th>洗面水栓<br> $w$<br>Bathroom faucet</th>
<th>浴槽水栓<br>湯はり<br> $b1$<br>Bathtub faucet hot water</th>
<th>浴槽自動<br>湯はり<br> $b2$<br>Automatic bath tub</th>
<th>浴槽水栓<br>差し湯<br> $ba1$<br>Bathtub faucet hot water</th>
<th>浴槽追焚<br> $ba2$<br>Bathtub memory</th>
</tr>
</thead>

<tbody>
<tr>
<th>$a_{std,u}$</th><td>0.0005</td><td>0.0024</td><td>0.0005</td><td>0.0000</td><td>0.0000</td><td>0.0000</td><td>0.0062</td>
</tr>
<tr>
<th>$b_{std,u}$</th><td>0.0028</td><td>0.0021</td><td>0.0028</td><td>-0.0027</td><td>-0.0024</td><td>-0.0027</td><td>0.0462</td>
</tr>
<tr>
<th>$c_{std,u}$</th><td>0.6818</td><td>0.7560</td><td>0.6818</td><td>0.9026</td><td>0.8885</td><td>0.9026</td><td>0.4001</td>
</tr>
</tbody>
</table>


In [ ]:
# 表 D.3 石油給湯機効率の回帰係
def table_d_3():
    tmp = [
        (0.0005, 0.0024, 0.0005, 0.0000, 0.0000, 0.0000, 0.0062),
        (0.0028, 0.0021, 0.0028, -0.0027, -0.0024, -0.0027, 0.0462),
        (0.6818, 0.7560, 0.6818, 0.9026, 0.8885, 0.9026, 0.4001)
    ]
    return tmp

$$\large
f_{hs} = \frac{(0.8669 \times e_{rtd} + 0.091)}{0.796}
$$

$e_{rtd}$：当該給湯機の効率(-) / Efficiency of the water heater concerned  
$f_{hs}$：当該給湯機に対する効率の補正係数(-) / Correction factor of efficiency for the relevant water heater 

In [ ]:
# 当該給湯機に対する効率の補正係数 (8)
def get_f_hs(e_rtd):
    """
    :param e_rtd: 当該給湯機の効率
    :return: 当該給湯機に対する効率の補正係数
    """
    return (0.8669 * e_rtd + 0.091) / 0.796

$$\large
e_{rtd} = e'_{rtd} -  0.081 
$$

$e_{rtd}$：当該給湯機の効率(-) / Efficiency of the water heater concerned  
$e'_{rtd}$：当該給湯機の JIS S 3031 の連続給湯効率試験方法又は湯沸効率試験方法に基づく「熱効率」(-) / "Thermal efficiency" based on JIS S 3031 continuous hot water supply efficiency test method or hot water efficiency test method of the water heater concerned 

In [ ]:
# 当該給湯機の効率
def get_e_rtd_default(hw_type):
    """
    :param hw_type: 給湯機／給湯温水暖房機の種類
    :return: 当該給湯機の効率
    """
    if hw_type in ['石油潜熱回収型給湯機', '石油潜熱回収型給湯温水暖房機']:
        return 0.819
    elif hw_type in ['石油従来型給湯機', '石油従来型給湯温水暖房機']:
        return 0.779
    else:
        raise ValueError(hw_type)


# 「エネルギーの使用の合理化に関する法律」に基づく「特定機器の性能の向上に関する製造事業者等の 判断の基準等」（ガス温水機器）
# に定義される「エネルギー消費効率」 から 当該給湯器の効率を取得
def get_e_rtd(e_dash_rtd):
    """
    :param e_dash_rtd: ：「エネルギーの使用の合理化に関する法律」に基づく「特定機器の性能の向上に関する製造事業者等の 判断の基準等」（ガス温水機器）に定義される「エネルギー消費効率」
    :return: 換算された当該給湯器の効率
    """
    return e_dash_rtd - 0.081

### C.5 １日当たりの太陽熱補正給湯熱負荷 / Solar heat correction hot water supply heat load per day


$$\large
L''_{k,d} = \sum_{t=0}^{23} L''_{k,d,t}
$$

$$\large
L''_{s,d} = \sum_{t=0}^{23} L''_{s,d,t}
$$

$$\large
L''_{w,d} = \sum_{t=0}^{23} L''_{w,d,t}
$$

$$\large
L''_{b1,d} = \sum_{t=0}^{23} L''_{b1,d,t}
$$

$$\large
L''_{b2,d} = \sum_{t=0}^{23} L''_{b2,d,t}
$$

$$\large
L''_{ba1,d} = \sum_{t=0}^{23} L''_{ba1,d,t}
$$

$$\large
L''_{ba2,d} = \sum_{t=0}^{23} L''_{ba2,d,t}
$$

$L''_{k,d}$：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load on a kitchen faucet per day at date $ d $  
$L''_{s,d}$：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at one day bathroom shower faucet at date $ d $  
$L''_{w,d}$：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load in a single day water faucet at date $ d $  
$L''_{b1,d}$：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub faucet hot water at date $ d $  
$L''_{b2,d}$：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub automatic bath heating on date $ d $  
$L''_{ba1,d}$：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub bathtub faucet at date $ d $  
$L''_{ba2,d}$：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d) / Solar heat correction hot water supply heat load at the time of one day bathtub remembrance at date $ d $  
$ L''_{k,d,t} $：日付$d$の時刻$t$における 1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load on kitchen faucet per hour at time $ t $ of date $ d $  
$ L''_{s,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load in bathroom shower faucet per hour at time $ t $ of date $ d $  
$ L''_{w,d,t} $：日付$d$の時刻$t$における 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load in the faucet per hour at time $ t $ of date $ d $  
$ L''_{b1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub faucet bathing per hour at time $ t $ of date $ d $  
$ L''_{b2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub automatic water heating per hour at time $ t $ of date $ d $  
$ L''_{ba1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub faucet breaking water per hour at time $ t $ of date $ d $  
$ L''_{ba2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h) / Solar heat correction hot water supply heat load at the time of bathtub remembrance per hour at time $ t $ of date $ d $  

In [ ]:
# 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
def get_L_dashdash_k_d(L_dashdash_k_d_t):
    """
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :return: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    """
    return np.sum(L_dashdash_k_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_s_d(L_dashdash_s_d_t):
    """
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_s_d_t.reshape((365, 24)), axis=1)


# 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_w_d(L_dashdash_w_d_t):
    """
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_w_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_b1_d(L_dashdash_b1_d_t):
    """
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_b1_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_b2_d(L_dashdash_b2_d_t):
    """
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_b2_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_ba1_d(L_dashdash_ba1_d_t):
    """
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_ba1_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_ba2_d(L_dashdash_ba2_d_t):
    """
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_ba2_d_t.reshape((365, 24)), axis=1)
